In [1]:
from typing import Annotated
from typing_extensions import TypeAliasType

from pydantic import BeforeValidator, AfterValidator, WrapValidator, PlainValidator, BaseModel, PlainSerializer, WithJsonSchema

class MyClass:
    pass

In [2]:
def dumb_validator(value, info):
    return value

def dumb_ser(value):
    return str(value)

In [15]:
MyClassType = TypeAliasType(
    'MyClassType',
    Annotated[
        int,
        PlainValidator(dumb_validator),
        PlainSerializer(dumb_ser),
        WithJsonSchema({"type": "monkey"})
    ]
)

In [16]:
class MyModel(BaseModel):
    a_thing: MyClassType
    b_thing: MyClassType

In [17]:
moo = MyModel(a_thing=5, b_thing=7)  # MyModel(a_thing=MyClassType, b_thing="hello")

In [18]:
moo.model_dump()

{'a_thing': '5', 'b_thing': '7'}

In [19]:
moo.model_dump_json()

'{"a_thing":"5","b_thing":"7"}'

In [20]:
moo.model_json_schema()

{'$defs': {'MyClassType': {'type': 'monkey'}},
 'properties': {'a_thing': {'$ref': '#/$defs/MyClassType'},
  'b_thing': {'$ref': '#/$defs/MyClassType'}},
 'required': ['a_thing', 'b_thing'],
 'title': 'MyModel',
 'type': 'object'}

In [21]:
class MyModelInt(BaseModel):
    a_thing: int
    b_thing: int

In [22]:
ioo = MyModelInt(a_thing=1, b_thing=2)

In [23]:
ioo.model_json_schema()

{'properties': {'a_thing': {'title': 'A Thing', 'type': 'integer'},
  'b_thing': {'title': 'B Thing', 'type': 'integer'}},
 'required': ['a_thing', 'b_thing'],
 'title': 'MyModelInt',
 'type': 'object'}